In [1]:
import sys
sys.path.insert(0, "../")

import math

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import shapely.geometry as geom
import geopandas as gpd

from weavingspace import TileUnit
from weavingspace import WeaveUnit
from weavingspace import Tiling
from weavingspace import tiling_utils

import shapely
shapely.__version__, np.__version__

('2.0.6', '2.0.2')

In [2]:
t = WeaveUnit(weave_type = "cube", strands = "abc|def|ghi", aspect = 3/4, spacing = 250)

In [3]:
# t = TileUnit(tiling_type = "cross", code = "4.8.8", n = 5, offset = 0) \
#     # .inset_prototile(0)
t = [WeaveUnit(weave_type = "cube", strands = "abc|def|ghi", aspect = a, spacing = 250) for a in np.linspace(0.7,0.8,51)]
# t.plot(r = 0, show_vectors = False, show_ids = False, show_prototile = False, show_reg_prototile = True)

# Scratch pad
A barebones notebook for testing basic operations of the package. The above code loads necessary libraries and allows for making a map.

In [4]:
# ak = gpd.read_file("data/imd-auckland-2018.gpkg")

In [5]:
# tile = TileUnit(tiling_type="square-colouring", 
#                 code="3.3.4.3.4", 
#                 spacing=500, 
#                 n=8, 
#                 crs=ak.crs) \
#                   .inset_prototile(10).inset_tiles(0)
# tile.plot(r=1)


In [6]:
# n = tile.tiles.shape[0]
# vars = dict(zip(list("abcdefghijkl")[:n], [x for x in ak.columns if "Rank" in x][:n]))
# cmaps = dict(zip(vars.values(), ["Reds", "Oranges", "Greens", "Blues", 
#                                  "Purples", "Greys", "YlOrBr", "PuRd", 
#                                  "YlOrRd", "BuGn", "BuPu", "YlGnBu" ][:n]))
# tiling = Tiling(tile, ak)
# map = tiling.get_tiled_map()
# fig = map.render(variables=vars, colourmaps=cmaps, use_ellipse=True)